In [1]:
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime as dt
from datetime import timedelta
import datetime
import json
import os
from sys import getsizeof
# import datetime
import random
# from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
# from py_vollib.black_scholes.greeks import analytical as greeks
# import time
# import ks_api_client
from kiteconnect import KiteConnect
# from kiteconnect import KiteTicker

In [5]:
current_datetime = dt(2021,5,17,9,30,0)
underlying_name =  'NIFTY'
current_datestring = current_datetime.strftime("%Y-%m-%d")

parent = os.path.dirname(os.getcwd())
historical_data_folder_name = 'historical data'
historical_data_folder_path = os.path.join(parent,historical_data_folder_name)

equity_data_folder_path = os.path.join(historical_data_folder_path,"Equity")
equity_file_paths = [os.path.join(equity_data_folder_path,f) \
    for f in os.listdir(equity_data_folder_path) \
        if os.path.isfile(os.path.join(equity_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0]==current_datestring)\
                ]

data_file_paths = equity_file_paths


print(data_file_paths)
df = pd.concat(map(pd.read_csv,data_file_paths))


['/Users/hg/Space/Code/AlgoTrader/historical data/Equity/2021-05-17.csv']


In [121]:
# Prepare Exchange Books
current_datetime = dt(2021,5,17,9,30,0)
underlying_name =  'NIFTY'
current_datestring = current_datetime.strftime("%Y-%m-%d")

parent = os.path.dirname(os.getcwd())
historical_data_folder_name = 'historical data'
historical_data_folder_path = os.path.join(parent,historical_data_folder_name)


fno_data_folder_path = os.path.join(historical_data_folder_path,"FNO")
fno_file_paths = [os.path.join(fno_data_folder_path,f) \
    for f in os.listdir(fno_data_folder_path) \
        if os.path.isfile(os.path.join(fno_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0]==current_datestring)\
                ]

equity_data_folder_path = os.path.join(historical_data_folder_path,"Equity")
equity_file_paths = [os.path.join(equity_data_folder_path,f) \
    for f in os.listdir(equity_data_folder_path) \
        if os.path.isfile(os.path.join(equity_data_folder_path,f)) \
            & (f.split(".")[0].split("_")[0]==current_datestring)\
                ]

data_file_paths = fno_file_paths
data_file_paths.extend(equity_file_paths)

print(data_file_paths)
df = pd.concat(map(pd.read_csv,data_file_paths))
print(f'1) {getsizeof(df)/1_000_000}')

df['expiry_datetime'] = pd.to_datetime(df['expiry_date']) + \
                        timedelta (hours=15, minutes =30)

print(f'2) {getsizeof(df)/1_000_000}')
df['timestamp'] = pd.to_datetime(df['timestamp'])

print(f'3) {getsizeof(df)/1_000_000}')

df.rename(columns={'Ticker':'ticker',
                'LTP':'ltp',
                'BuyPrice':'buy_price',
                'SellPrice':'sell_price'},
                inplace=True)

print(f'4) {getsizeof(df)/1_000_000}')

df = df[['ticker','ltp','buy_price','sell_price','underlying','strike','call_put','expiry_datetime','timestamp']]

print(f'4.5) {getsizeof(df)/1_000_000}')

df = df[(df['underlying']==underlying_name) | (df['underlying'].isnull())]

print(f'5) {getsizeof(df)/1_000_000}')

df_group = df.groupby(['ticker','timestamp'])
df['duplicate_serial'] = df_group[['ltp']].cumcount()

print(f'7) {getsizeof(df)/1_000_000}')

df['duplicate_count'] = df_group['ltp'].transform('size')

print(f'8) {getsizeof(df)/1_000_000}')

del df_group
df['duplicate_fraction'] = pd.to_timedelta(df['duplicate_serial'] / df['duplicate_count'], unit='s')

print(f'9) {getsizeof(df)/1_000_000}')
df['timestamp'] = df['timestamp'] + df['duplicate_fraction']

print(f'10) {getsizeof(df)/1_000_000}')

df = df[['ticker','ltp','buy_price','sell_price','underlying','strike','call_put','expiry_datetime','timestamp']]
print(f'11) {getsizeof(df)/1_000_000}')
df

a_df = df[['ticker','underlying','strike','call_put','expiry_datetime']]
a_df.drop_duplicates(inplace=True)
print(f'12) {getsizeof(a_df)/1_000_000}')

['/Users/hg/Space/Code/AlgoTrader/historical data/FNO/2021-05-17.csv', '/Users/hg/Space/Code/AlgoTrader/historical data/Equity/2021-05-17.csv']
1) 2622.718639
2) 2660.429279
3) 2339.888839
4) 2339.888839
4.5) 1209.744879
5) 1209.744879
7) 1247.455519
8) 1285.166159
9) 1322.876799
10) 1322.876799
11) 1209.744879
12) 0.119201


/var/folders/rw/d270qq7d70j5rl2p2xpt2yfr0000gn/T/ipykernel_42249/3087327181.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a_df.drop_duplicates(inplace=True)


In [125]:
#Create Kite object
def generate_access_token(broker_secret, request_token) -> str:

        kite = KiteConnect(api_key=broker_secret['kite_api_key'])

        data = kite.generate_session(request_token=request_token, \
            api_secret=broker_secret['kite_api_secret'])
        print("Token Generated")
        del kite 

        return data['access_token']

parent = os.path.dirname(os.getcwd())
file_path = os.path.join(parent,'broker_secret.json')

with open (file_path, "r") as openfile:
    broker_secret = json.load(openfile)

is_kite_access_token_available = False
kite_request_token='aI5JfQQTyttmqmceF4l3nlXz7fIxrz1V'

if not is_kite_access_token_available:
        broker_secret['kite_access_token'] = \
            generate_access_token(broker_secret,kite_request_token)

        with open (file_path, "w") as outfile:
            json.dump(broker_secret,outfile)

kite = KiteConnect(api_key=broker_secret['kite_api_key'])
kite.set_access_token(broker_secret['kite_access_token'])

kite.positions()



Token Generated


{'net': [], 'day': []}

In [126]:
nfo = pd.DataFrame(kite.instruments("NFO"))
nfo
    

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,14038786,54839,NIFTY22FEB15100CE,NIFTY,0.0,2022-02-24,15100.0,0.05,50,CE,NFO-OPT,NFO
1,14039042,54840,NIFTY22FEB15100PE,NIFTY,0.0,2022-02-24,15100.0,0.05,50,PE,NFO-OPT,NFO
2,14039298,54841,NIFTY22FEB15150CE,NIFTY,0.0,2022-02-24,15150.0,0.05,50,CE,NFO-OPT,NFO
3,14039554,54842,NIFTY22FEB15150PE,NIFTY,0.0,2022-02-24,15150.0,0.05,50,PE,NFO-OPT,NFO
4,14039810,54843,NIFTY22FEB15200CE,NIFTY,0.0,2022-02-24,15200.0,0.05,50,CE,NFO-OPT,NFO
...,...,...,...,...,...,...,...,...,...,...,...,...
69813,13338626,52104,WIPRO22FEBFUT,WIPRO,0.0,2022-02-24,0.0,0.05,800,FUT,NFO-FUT,NFO
69814,14887938,58156,WIPRO22MARFUT,WIPRO,0.0,2022-03-31,0.0,0.05,800,FUT,NFO-FUT,NFO
69815,17080322,66720,ZEEL22APRFUT,ZEEL,0.0,2022-04-28,0.0,0.05,3000,FUT,NFO-FUT,NFO
69816,13339906,52109,ZEEL22FEBFUT,ZEEL,0.0,2022-02-24,0.0,0.05,3000,FUT,NFO-FUT,NFO


In [130]:
a = ['NFO:NIFTY22FEB17200CE','NFO:NIFTY22FEB17200PE','NFO:NIFTY22FEB17300CE','NFO:NIFTY22FEB17300PE']
kite.ltp(a)

{'NFO:NIFTY22FEB17200CE': {'instrument_token': 13420290, 'last_price': 476.95},
 'NFO:NIFTY22FEB17200PE': {'instrument_token': 13420546, 'last_price': 311.85},
 'NFO:NIFTY22FEB17300CE': {'instrument_token': 13421314, 'last_price': 421.1},
 'NFO:NIFTY22FEB17300PE': {'instrument_token': 13421570, 'last_price': 356.15}}

In [164]:
#LTP
# a = 'NIFTY20MAY2114750CE.NFO'
a = ['NIFTY20MAY2114750CE.NFO','NIFTY20MAY2114750PE.NFO','NIFTY20MAY2114650CE.NFO']

current_datetime = dt(2021,5,17,14,4,32)
initiation_time = perf_counter()
# sleep(2)
slippage = perf_counter() - initiation_time
current_datetime_slippage_adj = current_datetime + timedelta(seconds=slippage)
current_datetime_slippage_adj

if type(a) == list:
    ltp_df = pd.DataFrame(a)
    ltp_df = ltp_df.merge(df,left_on=0, right_on='ticker')
    ltp_df = ltp_df[ltp_df['timestamp']<=current_datetime_slippage_adj].drop_duplicates(subset='ticker',keep='last')
    ltp_df = ltp_df[['ticker','ltp']]
    return_value = ltp_df
else:
    k = df[(df['ticker']==a)&(df['timestamp']<=current_datetime_slippage_adj)][['ltp']].iloc[-1][0]
    return_value = k
return_value

,ticker,ltp
17388,NIFTY20MAY2114750CE.NFO,191.2
39903,NIFTY20MAY2114750PE.NFO,48.7
62171,NIFTY20MAY2114650CE.NFO,269.4


In [173]:
#Quote
a = 'NIFTY20MAY2114750CE.NFO'


current_datetime = dt(2021,5,17,14,4,32)
initiation_time = perf_counter()
# sleep(2)
slippage = perf_counter() - initiation_time
current_datetime_slippage_adj = current_datetime + timedelta(seconds=slippage)
current_datetime_slippage_adj

k = df[(df['ticker']==a)&(df['timestamp']<=current_datetime_slippage_adj)][['buy_price','sell_price']].iloc[-1]
return_value = k
return_value


buy_price     191.3
sell_price    191.7
Name: 17388, dtype: float64

In [174]:
type(return_value['buy_price'])

numpy.float64

In [128]:
kite.quote('NFO:NIFTY22FEB17200CE')

{'NFO:NIFTY22FEB17200CE': {'instrument_token': 13420290,
  'timestamp': datetime.datetime(2022, 1, 28, 12, 43, 7),
  'last_trade_time': datetime.datetime(2022, 1, 28, 12, 42, 53),
  'last_price': 480,
  'last_quantity': 100,
  'buy_quantity': 41750,
  'sell_quantity': 38950,
  'volume': 1001250,
  'average_price': 456.55,
  'oi': 463000,
  'oi_day_high': 551700,
  'oi_day_low': 463000,
  'net_change': 0,
  'lower_circuit_limit': 0.05,
  'upper_circuit_limit': 857,
  'ohlc': {'open': 424.75, 'high': 495.75, 'low': 408.25, 'close': 372.45},
  'depth': {'buy': [{'price': 478.9, 'quantity': 100, 'orders': 1},
    {'price': 478.55, 'quantity': 200, 'orders': 1},
    {'price': 478.5, 'quantity': 200, 'orders': 2},
    {'price': 478.3, 'quantity': 50, 'orders': 1},
    {'price': 478.25, 'quantity': 200, 'orders': 1}],
   'sell': [{'price': 480.1, 'quantity': 100, 'orders': 1},
    {'price': 480.15, 'quantity': 100, 'orders': 2},
    {'price': 480.3, 'quantity': 100, 'orders': 1},
    {'price'

'2021-05-17'

In [7]:
#Slice data
all_df = pd.read_csv("data_df/Data 2022-02-17 12_35_51 team.csv")
all_df['current_datetime'] = pd.to_datetime(all_df['current_datetime'])
all_df['expiry_datetime'] = pd.to_datetime(all_df['expiry_datetime'])
# all_df.head()
# all_df.rename(columns={'candle_5_0_close_last':'candle_5_0_close',
#                         'candle_5_0_open_last':'candle_5_0_open',
#                         'candle_5_5_close_last':'candle_5_5_close',
#                         'candle_5_5_open_last':'candle_5_0_open'},
#                         inplace=True)


from_time = dt(2022,2,17,12,48,13)
till_time = dt(2022,2,17,15,50,1)

slice = all_df[(all_df['current_datetime']>=from_time) &\
   (all_df['current_datetime']<=till_time)].head(10)

slice[['current_datetime', 'current_pnl', 'strategy_pnl',
       'brokerage_pnl', 'max_pnl', 'trailing_pnl', 'current_ltp',
       'current_position', 'straddle_strike',
       'strangle_strike_low', 'position_entry_ltp', 'is_hedged', 'is_closed',
       'candle_t_2_open', 'candle_t_2_close', 'candle_t_1_open', 'candle_t_1_close']]

,current_datetime,current_pnl,strategy_pnl,brokerage_pnl,max_pnl,trailing_pnl,current_ltp,current_position,straddle_strike,strangle_strike_low,position_entry_ltp,is_hedged,is_closed,candle_t_2_open,candle_t_2_close,candle_t_1_open,candle_t_1_close
738,2022-02-17 12:48:13.896135,-250.0,-170.0,-80,0.0,-250.0,17372.55,straddle,17400.0,NaN,17395.75,True,False,17415.8,17402.6,17403.0,17395.7
739,2022-02-17 12:48:15.511616,2220.0,2340.0,-120,2220.0,0.0,17373.75,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
740,2022-02-17 12:48:16.602548,2190.0,2310.0,-120,2220.0,-30.0,17373.75,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
741,2022-02-17 12:48:17.708470,2265.0,2385.0,-120,2265.0,0.0,17373.75,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
742,2022-02-17 12:48:18.794689,2320.0,2440.0,-120,2320.0,0.0,17374.70,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
743,2022-02-17 12:48:19.890175,2360.0,2480.0,-120,2360.0,0.0,17375.05,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
744,2022-02-17 12:48:20.947439,2450.0,2570.0,-120,2450.0,0.0,17376.20,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
745,2022-02-17 12:48:22.054154,2400.0,2520.0,-120,2450.0,-50.0,17376.20,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
746,2022-02-17 12:48:23.127328,2440.0,2560.0,-120,2450.0,-10.0,17375.70,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7
747,2022-02-17 12:48:24.242847,2425.0,2545.0,-120,2450.0,-25.0,17375.25,NaN,NaN,NaN,NaN,True,False,17415.8,17402.6,17403.0,17395.7


In [11]:
p1 = pd.DataFrame({"tradingsymbol":{"0":"NIFTY2221717250PE","1":"NIFTY2221717400CE","2":"NIFTY2221717400PE","3":"NIFTY2221717550CE","4":"NIFTY2221717100PE"},"exchange":{"0":"NFO","1":"NFO","2":"NFO","3":"NFO","4":"NFO"},"instrument_token":{"0":13554178,"1":13571074,"2":13571330,"3":13591810,"4":13552642},"product":{"0":"MIS","1":"MIS","2":"MIS","3":"MIS","4":"MIS"},"quantity":{"0":100,"1":-100,"2":-100,"3":100,"4":0},"overnight_quantity":{"0":0,"1":0,"2":0,"3":0,"4":0},"multiplier":{"0":1,"1":1,"2":1,"3":1,"4":1},"average_price":{"0":3.2,"1":36.9166666667,"2":38.3166666667,"3":2.0,"4":0.0},"close_price":{"0":0,"1":0,"2":0,"3":0,"4":0},"last_price":{"0":3.25,"1":37.9,"2":28.0,"3":1.35,"4":1.25},"value":{"0":-320.0,"1":4970.0,"2":1565.0,"3":-175.0,"4":-25.0},"pnl":{"0":5.0,"1":1180.0,"2":-1235.0,"3":-40.0,"4":-25.0},"m2m":{"0":5.0,"1":1180.0,"2":-1235.0,"3":-40.0,"4":-25.0},"unrealised":{"0":5.0,"1":1180.0,"2":-1235.0,"3":-40.0,"4":-25.0},"realised":{"0":0,"1":0,"2":0,"3":0,"4":0},"buy_quantity":{"0":100,"1":200,"2":200,"3":200,"4":100},"buy_price":{"0":3.2,"1":30.525,"2":49.65,"3":2.0,"4":2.95},"buy_value":{"0":320,"1":6105,"2":9930,"3":400,"4":295},"buy_m2m":{"0":320,"1":6105,"2":9930,"3":400,"4":295},"sell_quantity":{"0":0,"1":300,"2":300,"3":100,"4":100},"sell_price":{"0":0.0,"1":36.9166666667,"2":38.3166666667,"3":2.25,"4":2.7},"sell_value":{"0":0.0,"1":11075.0,"2":11495.0,"3":225.0,"4":270.0},"sell_m2m":{"0":0.0,"1":11075.0,"2":11495.0,"3":225.0,"4":270.0},"day_buy_quantity":{"0":100,"1":200,"2":200,"3":200,"4":100},"day_buy_price":{"0":3.2,"1":30.525,"2":49.65,"3":2.0,"4":2.95},"day_buy_value":{"0":320,"1":6105,"2":9930,"3":400,"4":295},"day_sell_quantity":{"0":0,"1":300,"2":300,"3":100,"4":100},"day_sell_price":{"0":0.0,"1":36.9166666667,"2":38.3166666667,"3":2.25,"4":2.7},"day_sell_value":{"0":0.0,"1":11075.0,"2":11495.0,"3":225.0,"4":270.0}})
p2 = pd.DataFrame({"tradingsymbol":{"0":"NIFTY2221717250PE","1":"NIFTY2221717400CE","2":"NIFTY2221717400PE","3":"NIFTY2221717550CE","4":"NIFTY2221717100PE"},"exchange":{"0":"NFO","1":"NFO","2":"NFO","3":"NFO","4":"NFO"},"instrument_token":{"0":13554178,"1":13571074,"2":13571330,"3":13591810,"4":13552642},"product":{"0":"MIS","1":"MIS","2":"MIS","3":"MIS","4":"MIS"},"quantity":{"0":100,"1":-100,"2":-100,"3":100,"4":0},"overnight_quantity":{"0":0,"1":0,"2":0,"3":0,"4":0},"multiplier":{"0":1,"1":1,"2":1,"3":1,"4":1},"average_price":{"0":3.2,"1":36.9166666667,"2":38.3166666667,"3":2.0,"4":0.0},"close_price":{"0":0,"1":0,"2":0,"3":0,"4":0},"last_price":{"0":1.1,"1":3.05,"2":35.75,"3":27.95,"4":1.25},"value":{"0":-320.0,"1":4970.0,"2":1565.0,"3":-175.0,"4":-25.0},"pnl":{"0":5.0,"1":1180.0,"2":-1235.0,"3":-40.0,"4":-25.0},"m2m":{"0":5.0,"1":1180.0,"2":-1235.0,"3":-40.0,"4":-25.0},"unrealised":{"0":5.0,"1":1180.0,"2":-1235.0,"3":-40.0,"4":-25.0},"realised":{"0":0,"1":0,"2":0,"3":0,"4":0},"buy_quantity":{"0":100,"1":200,"2":200,"3":200,"4":100},"buy_price":{"0":3.2,"1":30.525,"2":49.65,"3":2.0,"4":2.95},"buy_value":{"0":320,"1":6105,"2":9930,"3":400,"4":295},"buy_m2m":{"0":320,"1":6105,"2":9930,"3":400,"4":295},"sell_quantity":{"0":0,"1":300,"2":300,"3":100,"4":100},"sell_price":{"0":0.0,"1":36.9166666667,"2":38.3166666667,"3":2.25,"4":2.7},"sell_value":{"0":0.0,"1":11075.0,"2":11495.0,"3":225.0,"4":270.0},"sell_m2m":{"0":0.0,"1":11075.0,"2":11495.0,"3":225.0,"4":270.0},"day_buy_quantity":{"0":100,"1":200,"2":200,"3":200,"4":100},"day_buy_price":{"0":3.2,"1":30.525,"2":49.65,"3":2.0,"4":2.95},"day_buy_value":{"0":320,"1":6105,"2":9930,"3":400,"4":295},"day_sell_quantity":{"0":0,"1":300,"2":300,"3":100,"4":100},"day_sell_price":{"0":0.0,"1":36.9166666667,"2":38.3166666667,"3":2.25,"4":2.7},"day_sell_value":{"0":0.0,"1":11075.0,"2":11495.0,"3":225.0,"4":270.0},"exchange:instrument_id":{"0":"NFO:NIFTY2221717250PE","1":"NFO:NIFTY2221717400CE","2":"NFO:NIFTY2221717400PE","3":"NFO:NIFTY2221717550CE","4":"NFO:NIFTY2221717100PE"},"qty_in_hand":{"0":100,"1":-100,"2":-100,"3":100,"4":0},"cash_flow_realized":{"0":-320.0,"1":4970.0,"2":1565.0,"3":-175.0,"4":-25.0},"cash_flow_unrealized":{"0":110.0,"1":-305.0,"2":-3575.0,"3":2795.0,"4":0.0}})
pnl = p2['cash_flow_realized'].sum() + \
                    p2['cash_flow_unrealized'].sum()
pnl

5040.0

In [17]:
display_col = ['tradingsymbol','quantity',
       'average_price',
       'last_price', 'value',
       'buy_quantity', 'buy_price', 'buy_value', 'buy_m2m', 'sell_quantity',
       'sell_price', 'sell_value', 'sell_m2m', 'day_buy_quantity',
       'day_buy_price', 'day_buy_value', 'day_sell_quantity', 'day_sell_price',
       'day_sell_value', 'exchange:instrument_id', 'qty_in_hand',
       'cash_flow_realized', 'cash_flow_unrealized']
p2[display_col]

,tradingsymbol,quantity,average_price,last_price,value,pnl,unrealised,realised,buy_quantity,buy_price,...,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value,exchange:instrument_id,qty_in_hand,cash_flow_realized,cash_flow_unrealized
0,NIFTY2221717250PE,100,3.200000,1.10,-320.0,5.0,5.0,0,100,3.200,...,100,3.200,320,0,0.000000,0.0,NFO:NIFTY2221717250PE,100,-320.0,110.0
1,NIFTY2221717400CE,-100,36.916667,3.05,4970.0,1180.0,1180.0,0,200,30.525,...,200,30.525,6105,300,36.916667,11075.0,NFO:NIFTY2221717400CE,-100,4970.0,-305.0
2,NIFTY2221717400PE,-100,38.316667,35.75,1565.0,-1235.0,-1235.0,0,200,49.650,...,200,49.650,9930,300,38.316667,11495.0,NFO:NIFTY2221717400PE,-100,1565.0,-3575.0
3,NIFTY2221717550CE,100,2.000000,27.95,-175.0,-40.0,-40.0,0,200,2.000,...,200,2.000,400,100,2.250000,225.0,NFO:NIFTY2221717550CE,100,-175.0,2795.0
4,NIFTY2221717100PE,0,0.000000,1.25,-25.0,-25.0,-25.0,0,100,2.950,...,100,2.950,295,100,2.700000,270.0,NFO:NIFTY2221717100PE,0,-25.0,0.0


In [51]:
p = pd.DataFrame({"tradingsymbol":{"0":"NIFTY2221717150PE","1":"NIFTY2221717300CE","2":"NIFTY2221717350PE","3":"NIFTY2221717450CE","4":"NIFTY2221717100PE","5":"NIFTY2221717250PE","6":"NIFTY2221717350CE","7":"NIFTY2221717400CE","8":"NIFTY2221717400PE","9":"NIFTY2221717550CE"},"exchange":{"0":"NFO","1":"NFO","2":"NFO","3":"NFO","4":"NFO","5":"NFO","6":"NFO","7":"NFO","8":"NFO","9":"NFO"},"instrument_token":{"0":13553154,"1":13554434,"2":13570818,"3":13579010,"4":13552642,"5":13554178,"6":13568514,"7":13571074,"8":13571330,"9":13591810},"product":{"0":"MIS","1":"MIS","2":"MIS","3":"MIS","4":"MIS","5":"MIS","6":"MIS","7":"MIS","8":"MIS","9":"MIS"},"quantity":{"0":100,"1":-100,"2":-100,"3":100,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0},"overnight_quantity":{"0":0,"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0},"multiplier":{"0":1,"1":1,"2":1,"3":1,"4":1,"5":1,"6":1,"7":1,"8":1,"9":1},"average_price":{"0":2.925,"1":47.3,"2":37.875,"3":2.8,"4":0.0,"5":0.0,"6":0.0,"7":0.0,"8":0.0,"9":0.0},"close_price":{"0":0,"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0},"last_price":{"0":3.35,"1":47.7,"2":46.6,"3":1.9,"4":2.2,"5":11.7,"6":20.55,"7":5.8,"8":82.3,"9":0.55},"value":{"0":-395.0,"1":4730.0,"2":4935.0,"3":-210.0,"4":-25.0,"5":-20.0,"6":-275.0,"7":1400.0,"8":-1225.0,"9":-65.0},"pnl":{"0":-60.0,"1":-40.0,"2":275.0,"3":-20.0,"4":-25.0,"5":-20.0,"6":-275.0,"7":1400.0,"8":-1225.0,"9":-65.0},"m2m":{"0":-60.0,"1":-40.0,"2":275.0,"3":-20.0,"4":-25.0,"5":-20.0,"6":-275.0,"7":1400.0,"8":-1225.0,"9":-65.0},"unrealised":{"0":-60.0,"1":-40.0,"2":275.0,"3":-20.0,"4":-25.0,"5":-20.0,"6":-275.0,"7":1400.0,"8":-1225.0,"9":-65.0},"realised":{"0":0,"1":0,"2":0,"3":0,"4":0,"5":0,"6":0,"7":0,"8":0,"9":0},"buy_quantity":{"0":200,"1":0,"2":100,"3":200,"4":100,"5":100,"6":100,"7":300,"8":300,"9":300},"buy_price":{"0":2.925,"1":0.0,"2":26.4,"3":2.8,"4":2.95,"5":3.2,"6":37.5,"7":32.25,"8":42.4,"9":1.5833333333},"buy_value":{"0":585.0,"1":0.0,"2":2640.0,"3":560.0,"4":295.0,"5":320.0,"6":3750.0,"7":9675.0,"8":12720.0,"9":475.0},"buy_m2m":{"0":585.0,"1":0.0,"2":2640.0,"3":560.0,"4":295.0,"5":320.0,"6":3750.0,"7":9675.0,"8":12720.0,"9":475.0},"sell_quantity":{"0":100,"1":100,"2":200,"3":100,"4":100,"5":100,"6":100,"7":300,"8":300,"9":300},"sell_price":{"0":1.9,"1":47.3,"2":37.875,"3":3.5,"4":2.7,"5":3.0,"6":34.75,"7":36.9166666667,"8":38.3166666667,"9":1.3666666667},"sell_value":{"0":190.0,"1":4730.0,"2":7575.0,"3":350.0,"4":270.0,"5":300.0,"6":3475.0,"7":11075.0,"8":11495.0,"9":410.0},"sell_m2m":{"0":190.0,"1":4730.0,"2":7575.0,"3":350.0,"4":270.0,"5":300.0,"6":3475.0,"7":11075.0,"8":11495.0,"9":410.0},"day_buy_quantity":{"0":200,"1":0,"2":100,"3":200,"4":100,"5":100,"6":100,"7":300,"8":300,"9":300},"day_buy_price":{"0":2.925,"1":0.0,"2":26.4,"3":2.8,"4":2.95,"5":3.2,"6":37.5,"7":32.25,"8":42.4,"9":1.5833333333},"day_buy_value":{"0":585.0,"1":0.0,"2":2640.0,"3":560.0,"4":295.0,"5":320.0,"6":3750.0,"7":9675.0,"8":12720.0,"9":475.0},"day_sell_quantity":{"0":100,"1":100,"2":200,"3":100,"4":100,"5":100,"6":100,"7":300,"8":300,"9":300},"day_sell_price":{"0":1.9,"1":47.3,"2":37.875,"3":3.5,"4":2.7,"5":3.0,"6":34.75,"7":36.9166666667,"8":38.3166666667,"9":1.3666666667},"day_sell_value":{"0":190.0,"1":4730.0,"2":7575.0,"3":350.0,"4":270.0,"5":300.0,"6":3475.0,"7":11075.0,"8":11495.0,"9":410.0}})
p['exchange:instrument_id'] = p['exchange'] \
                        + ":" + \
                        p['tradingsymbol']
kite_ltp = pd.DataFrame({"instrument_token":{"NFO:NIFTY2221717100PE":13552642,"NFO:NIFTY2221717150PE":13553154,"NFO:NIFTY2221717250PE":13554178,"NFO:NIFTY2221717300CE":13554434,"NFO:NIFTY2221717350CE":13568514,"NFO:NIFTY2221717350PE":13570818,"NFO:NIFTY2221717400CE":13571074,"NFO:NIFTY2221717400PE":13571330,"NFO:NIFTY2221717450CE":13579010,"NFO:NIFTY2221717550CE":13591810},"last_price":{"NFO:NIFTY2221717100PE":2.5,"NFO:NIFTY2221717150PE":3.65,"NFO:NIFTY2221717250PE":13.9,"NFO:NIFTY2221717300CE":43.85,"NFO:NIFTY2221717350CE":17.35,"NFO:NIFTY2221717350PE":51.3,"NFO:NIFTY2221717400CE":5.05,"NFO:NIFTY2221717400PE":89.15,"NFO:NIFTY2221717450CE":1.8,"NFO:NIFTY2221717550CE":0.5}}).reset_index()
p = pd.merge(p,kite_ltp[['index','last_price']],how='left',left_on='exchange:instrument_id',right_on='index',suffixes=("","_fetched"))
p['qty_in_hand'] = p['buy_quantity'] - \
                        p ['sell_quantity']
p['cash_flow_realized'] = \
    (p['sell_quantity']*p['sell_price']) - \
        (p['buy_quantity'] * p['buy_price'])
p['cash_flow_unrealized'] = \
    p['qty_in_hand'] * p['last_price_fetched']
pnl = round(p['cash_flow_realized'].sum() + \
    p['cash_flow_unrealized'].sum())
pnl

-120

In [48]:
p[['exchange:instrument_id','last_price_fetched']]

,exchange:instrument_id,last_price_fetched
0,NFO:NIFTY2221717150PE,3.65
1,NFO:NIFTY2221717300CE,43.85
2,NFO:NIFTY2221717350PE,51.30
3,NFO:NIFTY2221717450CE,1.80
4,NFO:NIFTY2221717100PE,2.50
5,NFO:NIFTY2221717250PE,13.90
6,NFO:NIFTY2221717350CE,17.35
7,NFO:NIFTY2221717400CE,5.05
8,NFO:NIFTY2221717400PE,89.15
9,NFO:NIFTY2221717550CE,0.50


In [49]:
kite_ltp

,index,instrument_token,last_price
0,NFO:NIFTY2221717100PE,13552642,2.50
1,NFO:NIFTY2221717150PE,13553154,3.65
2,NFO:NIFTY2221717250PE,13554178,13.90
3,NFO:NIFTY2221717300CE,13554434,43.85
4,NFO:NIFTY2221717350CE,13568514,17.35
5,NFO:NIFTY2221717350PE,13570818,51.30
6,NFO:NIFTY2221717400CE,13571074,5.05
7,NFO:NIFTY2221717400PE,13571330,89.15
8,NFO:NIFTY2221717450CE,13579010,1.80
9,NFO:NIFTY2221717550CE,13591810,0.50


In [46]:
p[['trading_symbol','last_price']]

,tradingsymbol,exchange,instrument_token,product,quantity,overnight_quantity,multiplier,average_price,close_price,last_price_x,...,sell_m2m,day_buy_quantity,day_buy_price,day_buy_value,day_sell_quantity,day_sell_price,day_sell_value,exchange:instrument_id,index,last_price_y
0,NIFTY2221717150PE,NFO,13553154,MIS,100,0,1,2.925,0,3.35,...,190.0,200,2.925000,585.0,100,1.900000,190.0,NFO:NIFTY2221717150PE,NFO:NIFTY2221717150PE,3.65
1,NIFTY2221717300CE,NFO,13554434,MIS,-100,0,1,47.300,0,47.70,...,4730.0,0,0.000000,0.0,100,47.300000,4730.0,NFO:NIFTY2221717300CE,NFO:NIFTY2221717300CE,43.85
2,NIFTY2221717350PE,NFO,13570818,MIS,-100,0,1,37.875,0,46.60,...,7575.0,100,26.400000,2640.0,200,37.875000,7575.0,NFO:NIFTY2221717350PE,NFO:NIFTY2221717350PE,51.30
3,NIFTY2221717450CE,NFO,13579010,MIS,100,0,1,2.800,0,1.90,...,350.0,200,2.800000,560.0,100,3.500000,350.0,NFO:NIFTY2221717450CE,NFO:NIFTY2221717450CE,1.80
4,NIFTY2221717100PE,NFO,13552642,MIS,0,0,1,0.000,0,2.20,...,270.0,100,2.950000,295.0,100,2.700000,270.0,NFO:NIFTY2221717100PE,NFO:NIFTY2221717100PE,2.50
5,NIFTY2221717250PE,NFO,13554178,MIS,0,0,1,0.000,0,11.70,...,300.0,100,3.200000,320.0,100,3.000000,300.0,NFO:NIFTY2221717250PE,NFO:NIFTY2221717250PE,13.90
6,NIFTY2221717350CE,NFO,13568514,MIS,0,0,1,0.000,0,20.55,...,3475.0,100,37.500000,3750.0,100,34.750000,3475.0,NFO:NIFTY2221717350CE,NFO:NIFTY2221717350CE,17.35
7,NIFTY2221717400CE,NFO,13571074,MIS,0,0,1,0.000,0,5.80,...,11075.0,300,32.250000,9675.0,300,36.916667,11075.0,NFO:NIFTY2221717400CE,NFO:NIFTY2221717400CE,5.05
8,NIFTY2221717400PE,NFO,13571330,MIS,0,0,1,0.000,0,82.30,...,11495.0,300,42.400000,12720.0,300,38.316667,11495.0,NFO:NIFTY2221717400PE,NFO:NIFTY2221717400PE,89.15
9,NIFTY2221717550CE,NFO,13591810,MIS,0,0,1,0.000,0,0.55,...,410.0,300,1.583333,475.0,300,1.366667,410.0,NFO:NIFTY2221717550CE,NFO:NIFTY2221717550CE,0.50


In [1]:
#Data Streaming using Websocket
import pandas as pd
import numpy as np
from time import perf_counter, perf_counter_ns
from time import sleep
from datetime import datetime as dt
from datetime import timedelta
import datetime
# import datetime
import random
from py_vollib.black_scholes.implied_volatility import implied_volatility as bs_iv
from py_vollib.black_scholes.greeks import analytical as greeks
import time
import ks_api_client
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker

tokens = []
def set_kite (kite_api_key=None, kite_access_token=None):

    kite = KiteConnect(api_key=kite_api_key)
    kite.set_access_token(kite_access_token)
    kws = KiteTicker(kite_api_key,kite_access_token)
    return kite, kws


def get_instruments_book (kite):
    nse = pd.DataFrame(kite.instruments("NSE"))
    nfo = pd.DataFrame(kite.instruments("NFO"))
    kite_instruments_book = nfo.append(nse)
    kite_instruments_book['expiry'] =  pd.to_datetime(kite_instruments_book['expiry'], 
                                            format='%Y-%m-%d') 
    kite_instruments_book['expiry_datetime'] = kite_instruments_book['expiry'] + \
                                        timedelta (hours=15, minutes =30)
    return kite_instruments_book


def tokenLookup(instrument_df,symbol_list):
    """Looks up instrument token for a given script from instrument dump"""
    token_list = []
    for symbol in symbol_list:
        token_list.append(int(instrument_df[instrument_df.tradingsymbol==symbol].instrument_token.values[0]))
    return token_list


def get_fno_instrument_id (instruments_df,strike, underlying, call_put, expiry_datetime) -> str:
        
            
    instrument_id = str(instruments_df[(instruments_df['name']==underlying)
            &(instruments_df['instrument_type']==call_put)
            &(instruments_df['expiry_datetime']==expiry_datetime)
            &(instruments_df['strike']==strike)]['instrument_token'].iloc[0])
    
    if not isinstance(instrument_id,str):
        instrument_id = str(instrument_id.iloc[0])

    return instrument_id


def get_multiple_fno_instrument_id (instruments_df, fno_df) -> pd.Series:
        # fno_df should have underlying, call_put, expiry_datetime and strike column

    instrument_id = fno_df.merge(instruments_df,how='left',\
            left_on=['underlying','call_put','expiry_datetime','strike'], 
            right_on=['name','instrument_type','expiry_datetime','strike'])\
            ['instrument_token'].astype(int)

    return instrument_id

def on_close(ws, code, reason):
    print("closed connection on close: {} {}".format(code, reason))


def on_error(ws, code, reason):
    print("closed connection on error: {} {}".format(code, reason))


def on_noreconnect(ws):
    print("Reconnecting the websocket failed")


def on_reconnect(ws, attempt_count):
    print("Reconnecting the websocket: {}".format(attempt_count))


def on_order_update(ws, data):
    print("order update: ", data)


def on_connect(ws,response):
    global tokens
    print("on connect: {}".format(response))
    # Callback on successful connect.
    # Subscribe to a list of instrument_tokens (RELIANCE and ACC here).
    #print("on connect: {}".format(response))
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL,tokens) # Set all token tick in `full` mode.
    #ws.set_mode(ws.MODE_FULL,[tokens[0]])  # Set one token tick in `full` mode.

counter = 0
combined_df = pd.DataFrame()
def on_ticks(ws,ticks):
    # Callback to receive ticks.
    # print("Ticks: {}".format(ticks))
    # print(ticks)
    # populate_df.delay(ticks)
    global counter
    global combined_df
    counter +=1
    tick_df = pd.DataFrame(ticks)[['instrument_token',\
        'last_price','volume','buy_quantity',\
        'sell_quantity','oi','timestamp','depth']]
    combined_df = combined_df.append(tick_df)
    # if dt.now().second == 0:
    #     combined_df.to_csv("tick_stream.csv",sep="|")
    #     print(f"saved,{counter}",end='\r')
    # else:
    print(f"         {counter}",end='\r')

def populate_df (ticks):
    global counter
    global combined_df
    counter +=1
    tick_df = pd.DataFrame(ticks)[['instrument_token',\
        'last_price','volume','buy_quantity',\
        'sell_quantity','oi','timestamp','depth']]
    combined_df = combined_df.append(tick_df)
    # if dt.now().second == 0:
    #     combined_df.to_csv("tick_stream.csv",sep="|")
    #     print(f"saved,{counter}",end='\r')
    # else:
    print(f"         {counter}",end='\r')


kite, kws = set_kite(kite_api_key="5ytarhiur9g1jebq",
    kite_access_token="oAwZpfJqmnb33FKt5wEczWCZmSdaTMKs")
instruments_df = get_instruments_book(kite)

next_expiry_datetime = instruments_df[\
                                    (instruments_df['name']=='NIFTY') 
                                    & (instruments_df['instrument_type'] == 'CE')]\
                                    ['expiry_datetime'].min()


# names = ['NIFTY']
names = ['NIFTY 50', 'INDIA VIX','NIFTY BANK', 'NIFTY','BANKNIFTY']
# names = ['NIFTY 50', 'INDIA VIX','NIFTY BANK']

tokens = list(instruments_df[(instruments_df['name']\
        .isin(names))&(instruments_df['expiry_datetime']==next_expiry_datetime)]\
            ['instrument_token'].astype(int))
print(type(tokens))
print(len(tokens))
# strike = 18_000.0
# underlying = 'NIFTY'
# expiry_datetime=dt(2022,1,27,15,30)
# fno_df = pd.DataFrame({'strike':[strike,strike,strike],\
#                         'underlying':[underlying,underlying,underlying],
#                         'expiry_datetime':[expiry_datetime,expiry_datetime,expiry_datetime],
#                         'call_put':['CE','PE','PE']})
# fno_df['tokens'] = get_multiple_fno_instrument_id(
#         instruments_df=instruments_df,
#         fno_df=fno_df)
# tokens = list(fno_df['tokens'])
# print(tokens)


<class 'list'>
464


In [2]:
# Assign the callbacks.
combined_df = pd.DataFrame()
kws.on_ticks = on_ticks
kws.on_connect = on_connect
kws.on_close = on_close
kws.on_error = on_error
kws.on_noreconnect = on_noreconnect
kws.on_reconnect = on_reconnect
kws.on_order_update = on_order_update

# kws.enable_reconnect(reconnect_interval=5, reconnect_tries=50)

# Infinite loop on the main thread. Nothing after this will run.
# You have to use the pre-defined callbacks to manage subscriptions.
# kws.connect(disable_ssl_verification=True)
kws.connect()

on connect: {"peer": "tcp4:13.127.118.127:443", "headers": {"date": "Mon, 24 Jan 2022 09:29:33 GMT", "connection": "upgrade", "upgrade": "websocket", "sec-websocket-accept": "h6FNntMirrG/aXf+PLjdol4KHBI="}, "version": 18, "protocol": null, "extensions": []}


Connection error: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
Connection closed: 1006 - connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


closed connection on error: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)
closed connection on close: 1006 connection was closed uncleanly (peer dropped the TCP connection without previous WebSocket closing handshake)


In [24]:
def connection_loss (**kwargs_decorator):
        d = None
        if 'd' in kwargs_decorator:
            d = kwargs_decorator.pop('d')
        def decorator_function (original_function):
            def wrapper(*args,**kwargs):
                print("Pre Execution")
                print(args[0].a,kwargs['another'])
                result = original_function(*args,**kwargs)
                print("Post Execution")
                return result
            return wrapper
        return decorator_function

class sample:
    def __init__ (self,a):
        self.a = a

    @connection_loss()
    def name(self,another=None):
        print(f"{self.a} Name {another}")

    

a = sample(2)
b = sample('kind')

a.name(another=3)
b.name(another="feel")



Pre Execution
2 3
2 Name 3
Post Execution
Pre Execution
kind feel
kind Name feel
Post Execution


In [18]:
a = None
not a

True